# M6 FAULT 10

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M6\FAULT_M6_10


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto=2

## RCOU
VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m6_fault10 = pd.read_csv("RCOU.csv")
rcou_m6_fault10 = rcou_m6_fault10.astype("float64")
rcou_m6_fault10 = rcou_m6_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m6_fault10 = rcou_m6_fault10[((rcou_m6_fault10['C9'] >= potenza) & (rcou_m6_fault10['C10'] >= potenza) & (rcou_m6_fault10['C11'] >= potenza) & (rcou_m6_fault10['C12'] >= potenza) & (rcou_m6_fault10['C13'] >= potenza) & (rcou_m6_fault10['C14']>= potenza))]
rcou_m6_fault10=rcou_m6_fault10.reset_index(drop=True)
display(rcou_m6_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,95775265.0,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0
1,95796452.0,1411.0,1447.0,1454.0,1403.0,1431.0,1428.0
2,95827847.0,1416.0,1477.0,1448.0,1447.0,1411.0,1481.0
3,95848531.0,1366.0,1540.0,1496.0,1421.0,1404.0,1510.0
4,95859847.0,1543.0,1376.0,1434.0,1496.0,1511.0,1416.0
...,...,...,...,...,...,...,...
7526,189014770.0,1661.0,1360.0,1649.0,1379.0,1470.0,1581.0
7527,189045755.0,1663.0,1365.0,1642.0,1397.0,1467.0,1590.0
7528,189056149.0,1594.0,1461.0,1661.0,1367.0,1402.0,1638.0
7529,189090843.0,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m6_fault10['TimeUS'][0]
max=rcou_m6_fault10['TimeUS'][len(rcou_m6_fault10)-1]
print(max)
print(min)

189119715.0
95775265.0


## ATT & XKF
### XKF
XKF utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m6_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m6_fault10 = xkf1_m6_fault10.astype("float64")
xkf1_m6_fault10 = xkf1_m6_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m6_fault10 = xkf1_m6_fault10[((xkf1_m6_fault10['TimeUS'] >= min) & (xkf1_m6_fault10['TimeUS'] <= max	))]
xkf1_m6_fault10 = xkf1_m6_fault10.reset_index(drop=True)
print(xkf1_m6_fault10)

           TimeUS  Roll  Pitch    Yaw
0      95783820.0  0.22   0.84  22.93
1      95793929.0  0.31   0.82  22.94
2      95804499.0  0.50   0.78  22.93
3      95815177.0  0.60   0.83  22.94
4      95825583.0  0.47   0.89  22.92
...           ...   ...    ...    ...
8479  189064391.0 -0.35  -1.60   8.61
8480  189078234.0 -0.40  -1.70   8.56
8481  189088483.0 -0.54  -1.71   8.55
8482  189098837.0 -0.47  -1.83   8.53
8483  189117546.0 -0.47  -1.76   8.44

[8484 rows x 4 columns]


### ATT
Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m6_fault10 = pd.read_csv("ATT.csv")
att_m6_fault10 = att_m6_fault10.astype("float64")
att_m6_fault10 = att_m6_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m6_fault10 = att_m6_fault10[((att_m6_fault10['TimeUS'] >= min) & (att_m6_fault10['TimeUS'] <= max	))]
att_m6_fault10=att_m6_fault10.reset_index(drop=True)
print(att_m6_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0      95785119.0    -0.07  0.22     -0.24   0.84   22.79  22.93
1      95795803.0    -0.10  0.31     -0.25   0.82   22.79  22.94
2      95806244.0    -0.12  0.50     -0.27   0.78   22.79  22.93
3      95816755.0    -0.13  0.60     -0.29   0.83   22.79  22.94
4      95826942.0    -0.14  0.47     -0.32   0.89   22.79  22.92
...           ...      ...   ...       ...    ...     ...    ...
8480  189065714.0     1.84 -0.35     -1.66  -1.60    3.38   8.61
8481  189079347.0     1.81 -0.40     -1.63  -1.70    3.38   8.56
8482  189089980.0     1.80 -0.54     -1.61  -1.71    3.38   8.55
8483  189100791.0     1.78 -0.47     -1.59  -1.83    3.38   8.53
8484  189118903.0     1.75 -0.47     -1.57  -1.76    3.39   8.44

[8485 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XKF corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
from numpy import empty

if len(att_m6_fault10)>len(xkf1_m6_fault10):
    to_add=att_m6_fault10[len(xkf1_m6_fault10):]
    att_m6_fault10=att_m6_fault10[:len(xkf1_m6_fault10)]

if len(xkf1_m6_fault10)>len(att_m6_fault10):
    to_add=xkf1_m6_fault10[len(att_m6_fault10):]

for idx,i in enumerate(att_m6_fault10['Roll']):
    if(xkf1_m6_fault10['Roll'][idx] != i):
        att_m6_fault10['Roll'][idx] = ((att_m6_fault10['Roll'][idx] + xkf1_m6_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m6_fault10['Pitch']):
    if(xkf1_m6_fault10['Pitch'][idx] != i):
        att_m6_fault10['Pitch'][idx] = ((att_m6_fault10['Pitch'][idx] + xkf1_m6_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m6_fault10['Yaw']):
    if(xkf1_m6_fault10['Yaw'][idx] != i):
        att_m6_fault10['Yaw'][idx] = ((att_m6_fault10['Yaw'][idx] + xkf1_m6_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m6_fault10=pd.concat([att_m6_fault10,to_add])

print(att_m6_fault10)

           TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw     Yaw
0      95785119.0    -0.07  0.220     -0.24  0.840   22.79  22.930
1      95795803.0    -0.10  0.310     -0.25  0.820   22.79  22.940
2      95806244.0    -0.12  0.500     -0.27  0.780   22.79  22.930
3      95816755.0    -0.13  0.600     -0.29  0.830   22.79  22.940
4      95826942.0    -0.14  0.470     -0.32  0.890   22.79  22.920
...           ...      ...    ...       ...    ...     ...     ...
8480  189065714.0     1.84 -0.375     -1.66 -1.650    3.38   8.585
8481  189079347.0     1.81 -0.470     -1.63 -1.705    3.38   8.555
8482  189089980.0     1.80 -0.505     -1.61 -1.770    3.38   8.540
8483  189100791.0     1.78 -0.470     -1.59 -1.795    3.38   8.485
8484  189118903.0     1.75 -0.470     -1.57 -1.760    3.39   8.440

[8485 rows x 7 columns]


## IMU
VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m6_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m6_fault10 = imu_0_m6_fault10.astype("float64")
imu_0_m6_fault10 = imu_0_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m6_fault10 = imu_0_m6_fault10[((imu_0_m6_fault10['TimeUS'] >= min) & (imu_0_m6_fault10['TimeUS'] <= max))]

imu_1_m6_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m6_fault10 = imu_1_m6_fault10.astype("float64")
imu_1_m6_fault10 = imu_1_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m6_fault10 = imu_1_m6_fault10[((imu_1_m6_fault10['TimeUS'] >= min) & (imu_1_m6_fault10['TimeUS'] <= max))]

imu_2_m6_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m6_fault10 = imu_2_m6_fault10.astype("float64")
imu_2_m6_fault10 = imu_2_m6_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m6_fault10 = imu_2_m6_fault10[((imu_2_m6_fault10['TimeUS'] >= min) & (imu_2_m6_fault10['TimeUS'] <= max))]

imu_m6_fault10 = pd.concat((imu_0_m6_fault10, imu_1_m6_fault10, imu_2_m6_fault10))
imu_m6_fault10=imu_m6_fault10.groupby(imu_m6_fault10.TimeUS, as_index=False).mean()

display(imu_m6_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,95777043.0,0.602359,-0.056894,-0.000354,-0.039858,1.590671,-9.782290
1,95780532.0,0.620342,-0.147095,0.034039,0.125093,0.755188,-9.501524
2,95782925.0,-0.085625,-0.118474,0.042167,0.185636,-0.309531,-9.536599
3,95784938.0,-0.370824,0.018751,0.013881,0.083057,-0.263599,-9.828410
4,95787735.0,-0.001924,0.109419,-0.019396,-0.036998,0.582240,-10.132694
...,...,...,...,...,...,...,...
32248,189100550.0,0.071910,-0.026657,-0.051577,-0.556881,0.403775,-10.109230
32249,189107000.0,-0.230953,0.107526,-0.084279,-0.836197,0.923446,-10.389923
32250,189109336.0,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483
32251,189116807.0,-0.164212,0.141485,-0.067597,-0.956638,0.871506,-10.755723


## UNIONE DELLE TABELLE
Vengono unite le tabelle relative a IMU, ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m6_fault10 = pd.merge_ordered(imu_m6_fault10,att_m6_fault10)
m6_fault10=m6_fault10.fillna(method="ffill").fillna(method="bfill")
m6_fault10 = pd.merge_ordered(m6_fault10, rcou_m6_fault10)
m6_fault10=m6_fault10.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m6_fault10["TimeUS"] = m6_fault10["TimeUS"] - m6_fault10.iloc[0]["TimeUS"]
m6_fault10["TimeUS"] = m6_fault10["TimeUS"].astype(int)
m6_fault10["Guasto"] = guasto

In [ ]:
last_cell = m6_fault10.iloc[-1, m6_fault10.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [ ]:
m6_fault10_final = pd.merge_ordered(m6_fault10,df)
m6_fault10_final=m6_fault10_final.fillna(method="ffill").fillna(method="bfill")
m6_fault10_final = m6_fault10_final[m6_fault10_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [14]:
from datetime import timedelta
m6_fault10_final=m6_fault10_final.reset_index(drop=True)
m6_fault10_final['TimeUS'] = pd.to_datetime(m6_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m6_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,0.602359,-0.056894,-0.000354,-0.039858,1.590671,-9.782290,-0.07,0.22,-0.24,0.840,22.79,22.930,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2.0
1,00:00:00.002857,0.602359,-0.056894,-0.000354,-0.039858,1.590671,-9.782290,-0.07,0.22,-0.24,0.840,22.79,22.930,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2.0
2,00:00:00.005714,0.620342,-0.147095,0.034039,0.125093,0.755188,-9.501524,-0.07,0.22,-0.24,0.840,22.79,22.930,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2.0
3,00:00:00.008571,-0.085625,-0.118474,0.042167,0.185636,-0.309531,-9.536599,-0.07,0.22,-0.24,0.840,22.79,22.930,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2.0
4,00:00:00.011428,-0.370824,0.018751,0.013881,0.083057,-0.263599,-9.828410,-0.07,0.22,-0.24,0.840,22.79,22.930,1389.0,1443.0,1402.0,1431.0,1367.0,1463.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32668,00:01:33.332476,-0.230953,0.107526,-0.084279,-0.836197,0.923446,-10.389923,1.78,-0.47,-1.59,-1.795,3.38,8.485,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2.0
32669,00:01:33.335333,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483,1.78,-0.47,-1.59,-1.795,3.38,8.485,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2.0
32670,00:01:33.338190,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483,1.78,-0.47,-1.59,-1.795,3.38,8.485,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2.0
32671,00:01:33.341047,0.201615,-0.015658,-0.068950,-0.539806,1.456464,-10.613483,1.78,-0.47,-1.59,-1.795,3.38,8.485,1575.0,1480.0,1647.0,1385.0,1361.0,1663.0,2.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [15]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [16]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m6_fault10_final)/350)):
        V1=m6_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [17]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_31040\1036771359.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_31040\1036771359.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.028497,0.018888,4.194314,0.140165,92.0,0.0,5.954716,5.680965,-0.011499,0.014602,-0.499496,0.121212,0.0,88.0,19.040100,3.933249,0.129697,0.005434,-0.910038,0.149131,69.0,63.0,1.185441,0.362920,-0.075775,0.057781,0.209573,0.251711,0.0,106.0,18.259877,14.502475,0.325590,0.065944,3.286183,0.414446,102.0,104.0,12.722840,12.048912,-10.262300,0.488720,-0.134301,10.286016,0.0,115.0,103.358946,22.963991,0.152849,0.698996,-0.971515,0.848745,95.0,100.0,4.784579,4.380033,-0.120427,0.954856,-1.454061,0.983178,0.0,110.0,111.083317,4.119011,25.480484,5.531387,-1.161600,25.588488,110.0,109.0,4.220248,-1.390539,1560.133903,3903.739162,1.701250,1561.380931,84.0,82.0,1470.745283,996.391071,1474.131054,2496.108490,-0.754414,1474.975038,99.0,98.0,852.849836,770.503643,1536.028490,2193.176329,1.026951,1536.740203,82.0,87.0,1317.904384,818.881846,1502.099715,2543.192886,-0.560373,1502.943612,84.0,82.0,1337.995611,1004.901225,1494.732194,3162.013789,-0.521189,1495.786528,109.0,105.0,2239.559929,768.592655,1542.413105,2298.660285,0.617778,1543.155954,116.0,115.0,1928.140162,919.724791,2
1,1.0,-0.028396,0.018838,4.208332,0.139966,82.0,0.0,6.145259,5.815276,-0.011204,0.014591,-0.499033,0.121139,0.0,92.0,18.993719,4.073417,0.129792,0.005422,-0.902461,0.149172,73.0,62.0,1.194850,0.559475,-0.075341,0.057683,0.214159,0.251387,0.0,100.0,18.371931,15.070126,0.326302,0.065934,3.272318,0.414994,100.0,103.0,12.938945,12.177540,-10.260591,0.488356,-0.126543,10.284293,0.0,102.0,102.891941,23.606158,0.148097,0.704955,-0.976960,0.851402,91.0,96.0,4.656365,4.375201,-0.119801,0.952274,-1.449576,0.981795,0.0,113.0,112.968610,4.074598,25.493920,5.579175,-1.163262,25.602799,103.0,102.0,4.233476,-1.326368,1560.338068,3907.289943,1.703961,1561.586077,76.0,75.0,1412.550079,927.878842,1473.835227,2519.801832,-0.753592,1474.687398,85.0,84.0,949.556509,895.280965,1536.159091,2192.931883,1.033891,1536.870669,98.0,103.0,1336.322900,825.938896,1501.923295,2546.902932,-0.564391,1502.768529,100.0,98.0,1356.862420,1026.057854,1494.744318,3153.056948,-0.514002,1495.795667,96.0,93.0,2180.677223,723.875620,1542.377841,2292.549137,0.624082,1543.118738,108.0,110.0,2074.436427,854.067567,2
2,2.0,-0.028669,0.018810,4.218814,0.139925,90.0,0.0,6.199758,5.817502,-0.010613,0.014672,-0.504358,0.121423,0.0,92.0,19.051325,4.306219,0.129981,0.005419,-0.896582,0.149328,61.0,49.0,1.237279,0.657356,-0.075261,0.057521,0.223503,0.251043,0.0,108.0,18.358334,14.866145,0.327100,0.065972,3.251098,0.415668,102.0,105.0,13.108666,12.578030,-10.258342,0.488753,-0.121367,10.282070,0.0,105.0,102.615987,24.400641,0.143371,0.710835,-0.982790,0.854035,93.0,98.0,4.429762,4.376361,-0.119178,0.949705,-1.445087,0.980417,0.0,115.0,114.837276,3.998952,25.507280,5.626332,-1.165469,25.61702

## Output
Salviamo la tabella ottenuta in un file csv

In [18]:
path_file = path_file.replace(r"M6\FAULT_M6_10", "")
os.chdir(path_file)
df_merged.to_csv('m6_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
